# Processamento de Texto e Redução de Dimensionalidade com Feature Hashing

## Analise de Algoritmos e Estruturas de Dados - Profa. Lilian Berton
@author: Willian Dihanster Gomes de Oliveira

Referencias:
 - https://kavita-ganesan.com/hashingvectorizer-vs-countvectorizer/#.YLAX7qhKjIV
 - https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py


### 1 - Imports e Leitura do Dataset
Aqui faremos os imports e leitura dos dataset, que é esta disponível diretamente no sklearn que textos sobre diversos assuntos. No caso, será selecionado um subset balanceado de treino e teste com textos sobre 'eletronics' e 'space'.

In [1]:
import time
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [2]:
categories = ['sci.electronics', 'sci.space']

train = fetch_20newsgroups(
    subset='train', 
    categories=categories,
    remove=('headers', 'footers', 'quotes')
)
test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    remove=('headers', 'footers', 'quotes')
)

#### 1.1- Exemplo de texto sobre 'eletronics'

In [3]:
train.data[20]

"\n\n\nCan I suggest the University of Western Australia in Perth.\nThe weathers great, the people are great and our Electronic Engineering department is great.\nI am a first year student here ... so I don't know much about what projects but I do know they have a good reputation in the fields of dsp and communications.  Ever heard of QPSX?  The people who own are ex-UWA ... so that gives an indication of what the department is like.\n\nFor more information\nemail: yianni@swanee.ee.uwa.edu.au\nwith the above request and he should be able to tell some more info\n\nor write\n\nDepartment of Electrical and Electronic Engineering\nUniversity of Western Australia\nStirling Highway\nCRAWLEY 6009\nWestern Australia\nAustralia\n\n\nYours\nMark\n\nmtearle@tartarus.uwa.edu.au"

#### 1.2 - Exemplo de texto sobre 'space'

In [4]:
train.data[3]

'\n[Excellent discussion of DC-X landing techniques by Henry deleted]\n\n\nThe DC-X will not take of horizontally.  It takes of vertically. \n\n\nFor several reasons.  Vertical landings don\'t require miles of runway and limit\nnoise pollution.  They don\'t require wheels or wings.  Just turn on the engines\nand touch down.  Of course, as Henry pointed out, vetical landings aren\'t quite\nthat simple.\n\n\nWell, to be blunt, yes.  But at least you\'re learning.\n\n\nThe Soyuz vehicles use parachutes for the descent and then fire small rockets\njust before they hit the ground.  Parachutes are, however, not especially\npractical if you want to reuse something without much effort.  The landings\nare also not very comfortable.  However, in the words of Georgy Grechko,\n"I prefer to have bruises, not to sink."\n\n'

### 2 - Aplicação do HashingVectorizer
Aqui faremos a aplicação do HashingVectorizer. Note que o principal parâmetro aqui é N, o valor que consideraremos para o tamanho do nosso espaço de features. Podemos definir qualquer valor, porém, levando em conta que valores muito pequenos, possivelmente gerará mais colisões e com isso, mais erros.

Além disso, executaremos um total de 10 vezes e tiraremos a média do tempo gasto, e algumas métricas finais para comparação dos métodos.

In [5]:
tempos = []
N = 1000

for i in range(10):
    inicio = time.time()
    
    # Inicia o vectorizer
    vectorizer = HashingVectorizer(n_features=N, alternate_sign=False)
    
    # Aplica o vectorizer no dataset de treino e teste
    X_train = vectorizer.fit_transform(train.data)
    y_train = train.target
    
    X_test = vectorizer.transform(test.data)
    y_test = test.target
    
    # Inicia e treina o classificador Multinomial Naive Bayes
    clf = MultinomialNB(alpha=.01)
    clf.fit(X_train, y_train)
    MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
    
    # Faz as predicoes
    y_pred = clf.predict(X_test)
    
    # Salva o tempo gasto nesta iteracao
    fim = time.time()
    tempos.append(fim-inicio)

# Printa o ultim o resultado da Classificacao e o tempo medio gasto
print(classification_report(y_test, y_pred))
print(f'Tempo medio com o HashingVectorizer = {round(sum(tempos)/len(tempos), 15)}s')        

              precision    recall  f1-score   support

           0       0.86      0.73      0.79       393
           1       0.76      0.88      0.82       394

    accuracy                           0.80       787
   macro avg       0.81      0.80      0.80       787
weighted avg       0.81      0.80      0.80       787

Tempo medio com o HashingVectorizer = 0.257513380050659s


### 3 - Aplicação do CountingVectorizer
Aqui faremos o uso do algoritmo CountingVectorizer, seguindo a mesma ideia de execução e avaliação do algoritmo anterior. Note que também podemos definir o valor de N, que definir o máximo de colunas utilizadas, porém, o algoritmo ainda sim irá calcular todo o dicionário de palavras e então considerar as N mais populares, o que pode levar mais tempo que o método anterior mesmo com N igual ou menor.

In [6]:
tempos = []
N = None

for i in range(10):
    inicio = time.time()

    # Inicia o vectorizer
    vectorizer = CountVectorizer(max_features=N)

    # Aplica o vectorizer no dataset de treino e teste
    X_train = vectorizer.fit_transform(train.data)
    y_train = train.target
    
    X_test = vectorizer.transform(test.data)
    y_test = test.target
    
    # Inicia e treina o classificador Multinomial Naive Bayes
    clf = MultinomialNB(alpha=.01)
    clf.fit(X_train, y_train)
    MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
    
    # Faz as predicoes
    y_pred = clf.predict(X_test)
    
    # Salva o tempo gasto nesta iteracao
    fim = time.time()
    tempos.append(fim-inicio)

# Printa o ultim o resultado da Classificacao e o tempo medio gasto
print(classification_report(y_test, y_pred))
print(f'Tempo medio com o CountVectorizer = {round(sum(tempos)/len(tempos), 15)}s')  

              precision    recall  f1-score   support

           0       0.95      0.88      0.91       393
           1       0.88      0.95      0.92       394

    accuracy                           0.91       787
   macro avg       0.91      0.91      0.91       787
weighted avg       0.91      0.91      0.91       787

Tempo medio com o CountVectorizer = 0.38299834728241s


### 4 - Conclusões
Hashing Vectorizer pode ser uma boa opção quando a prioridade é tempo e/ou menos gastos de memória, e é aceitável colisões/errar mais e quando não é necessário sabermos as palavras utilizadas. <br>
Counting Vectorizer pode ser mais indicado quando se busca melhores resultados (em termos de acurácia) e/ou se saber qual palavra foi utilizada será útil, além de ser possível aceitar o tempo maior e o uso de memória a mais.